## NYC Citi Bike Data Analysis

Data from: https://citibikenyc.com/system-data

### Prerequisites

Before running, download all the zip files from this URL for the year 2024: https://s3.amazonaws.com/tripdata/index.html

In [ ]:
from pathlib import Path
from zipfile import ZipFile

import duckdb

In [ ]:
duckdb.install_extension("spatial")
duckdb.install_extension("h3", repository="community")
duckdb.install_extension("zipfs", repository="community")

duckdb.load_extension("spatial")
duckdb.load_extension("h3")
duckdb.load_extension("zipfs")

In [ ]:
DOWNLOAD_PATH = Path("data")

In [ ]:
zip_file_paths = sorted(
    f"zip://{zip_file_path}/{name}"
    for zip_file_path in DOWNLOAD_PATH.glob("*.zip")
    for name in ZipFile(zip_file_path).namelist()
)

In [ ]:
H3_RES = 9

duckdb.sql(f"""
WITH raw_trips AS (
    SELECT
        ride_id,
        start_lat::FLOAT as start_lat,
        start_lng::FLOAT as start_lng,
        end_lat::FLOAT as end_lat,
        end_lng::FLOAT as end_lng,
        started_at::TIMESTAMP as started_at
    FROM read_csv(
        {zip_file_paths},
        header=True,
        all_varchar=True
    )
),
parsed_trips AS (
    SELECT
        ride_id,
        h3_latlng_to_cell(start_lat, start_lng, {H3_RES}) as start_h3,
        h3_latlng_to_cell(end_lat, end_lng, {H3_RES}) as end_h3,
        CASE
            WHEN HOUR(started_at) BETWEEN 6 and 10 THEN 'morning'
            WHEN HOUR(started_at) BETWEEN 16 and 20 THEN 'afternoon'
        END as day_part
    FROM raw_trips
    WHERE
        (
            HOUR(started_at) BETWEEN  6 and 10 OR
            HOUR(started_at) BETWEEN 16 and 20
        ) AND
        start_lng BETWEEN -74.1 AND -73.8 AND
        end_lng   BETWEEN -74.1 AND -73.8 AND
        start_lat BETWEEN  40.6 AND  40.9 AND
        end_lat   BETWEEN  40.6 AND  40.9
),
morning_starts AS (
    SELECT
        start_h3 as h3,
        COUNT(*) as morning_starts
    FROM parsed_trips
    WHERE day_part = 'morning'
    GROUP BY start_h3
),
morning_ends AS (
    SELECT
        end_h3 as h3,
        COUNT(*) as morning_ends
    FROM parsed_trips
    WHERE day_part = 'morning'
    GROUP BY end_h3
),
afternoon_starts AS (
    SELECT
        start_h3 as h3,
        COUNT(*) as afternoon_starts
    FROM parsed_trips
    WHERE day_part = 'afternoon'
    GROUP BY start_h3
),
afternoon_ends AS (
    SELECT
        end_h3 as h3,
        COUNT(*) as afternoon_ends
    FROM parsed_trips
    WHERE day_part = 'afternoon'
    GROUP BY end_h3
)
SELECT
    h3,
    COALESCE(morning_starts, 0) AS morning_starts,
    COALESCE(morning_ends, 0) AS morning_ends,
    COALESCE(afternoon_starts, 0) AS afternoon_starts,
    COALESCE(afternoon_ends, 0) AS afternoon_ends,
    ST_GeomFromText(h3_cell_to_boundary_wkt(h3)) AS geometry
FROM morning_starts ms
FULL OUTER JOIN morning_ends me USING (h3)
FULL OUTER JOIN afternoon_starts AS ats USING (h3)
FULL OUTER JOIN afternoon_ends AS ate USING (h3)
WHERE h3 IS NOT NULL
ORDER BY h3
""").to_csv("nyc_bike_trips.csv.gz")